In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def get_scores(ops,gts):
    sc = 0
    for k in range(len(ops)):
        try:
            cscore = sentence_bleu(gts[k], ops[k], smoothing_function=SmoothingFunction().method4)
            print(gts[k])
            print(ops[k])
        except:
            cscore = 0
        sc += cscore

    print("Bleu: ", sc/len(ops))

In [ ]:
from EncoderDecoder import EncoderDecoder, EncoderCNN, DecoderRNN
from data_utils import Img2LatexDataset, load_img
from train_model import *
from torch.utils.data import DataLoader

model = load_model("./models/model_4ep.pt")
model.eval()

dataset = Img2LatexDataset("../data/SyntheticData/images/", "../data/SyntheticData/test.csv", tokens=model.decoder.vocab, 
                           token_to_idx=model.decoder.vocab_dict)
loader = DataLoader(dataset, batch_size=1, shuffle=True)

batch = next(iter(loader))
print(f"Images shape: {batch[0].shape}, formulas shape: {batch[1].shape}")

In [ ]:
def label_to_string(label):
    token_list = []
    for tok in label:
        if tok == model.decoder.vocab_dict["<sos>"] or tok == model.decoder.vocab_dict["<pad>"]:
            continue
        if tok == model.decoder.vocab_dict["<eos>"] or tok == model.decoder.vocab_dict["<pad>"]:
            break
        token_list.append(model.decoder.vocab[tok])

    return " ".join(token_list)

overall = 0

counted = 1

for batch in loader:
    imgs, labels = batch
    preds = []

    for i in range(len(labels)):
        preds.append(" ".join(model(imgs[i])))  
    counted += 1
        
    ground_truths = [label_to_string(labels[i]) for i in range(len(labels))]
    bleu = get_scores(preds, ground_truths)

    if counted % 5 == 0:
        print("Macro Bleu : ", overall/counted)

In [ ]:
import pandas as pd
import os

def read_and_pred(img_path, model):
    print(f"New func")
    img = load_img(img_path)
    if img.shape[0] == 1:
        img = img.repeat(3, axis=0)
    print(img.shape)
        
    pred = model(img)
    print(pred)
    return pred

model = load_model("./models/model_4ep.pt")
dataset_dir = "../data/sample_sub.csv"
imgs_dir = "../data/HandwrittenData/images/test/"
preds_df = pd.read_csv(dataset_dir)

preds_df["formula"] = preds_df["image"].apply(lambda x: read_and_pred(os.path.join(imgs_dir, x), model))
preds_df.to_csv(dataset_dir)